In [1]:
import sys
import requests
from dotenv import load_dotenv
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3

In [2]:
smart_contract = initContract()

Contract Address 0x5A82ddFE2E00a3FCcdEA2cD10e110B72e4535677


In [4]:
def createNewMessage():
    title = input("Enter a message title: ")
    message = input("Enter a message body: ")
    tags = input("Enter a comma seperated lsit of tags.")
    
    message_object = {
            "title": title,
            "message": message,
            "tags": tags,
    }

    return message_object

In [5]:
def postMessage():
    message_object = createNewMessage() # Create New Message Object
    json_data = convertDataToJSON(message_object) # Convert essage object to JSON
    message_uri = pinJSONtoIPFS(json_data) # Pin JSON to IPFS and get Hash
    
    tx_hash = smart_contract.functions.postMessage(str(message_uri)).transact(
        {"from": w3.eth.accounts[0]}
    )

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [6]:
def getLatestMessage():
    ipfs_hash = smart_contract.functions.getLatestMessage().call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [7]:
def getMessageByID(message_id):
    ipfs_hash = smart_contract.functions.messages(message_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [10]:
option = input("Would you like to get or post a message?")

if option == "post":
    postMessage()

if option == "get":
    message_id = input("Enter a message ID to retive the message. Hit enter to get the latest.")

    if (message_id):
        print(getMessageByID(int(message_id)))
    else:
        print(getLatestMessage())

Would you like to get or post a message?post
Enter a message title: Looking to buy asparagus
Enter a message body: I need 8 fully grow asparagus plants to save my mother, please help. Will pay top dollar.
Enter a comma seperated lsit of tags.asparagus
